# Building a Predictor

Now that our data is in here and working the next step is to build the MODEL so we can put it to practice

In [1]:
# Importing PyTorch libraries and the traintest split function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Importing libraries for plotting, data, and math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing Data and Splitting

First step of any machine learning model is to import the data you want to work with and split it into our training and testing data. We will be using the train_test_split function from sklearn to do this.

In [2]:
# Importing our data
data = pd.read_csv('ufc_fights.csv')
data.head()

,HeightDiff,WeightDiff,ReachDiff,WinDiff,Winner
0,2.0,7.0,0.0,0.062500,0
1,-4.0,1.0,-5.0,0.010000,1
2,0.0,-9.0,-1.0,-0.142857,1
3,0.0,11.0,0.0,0.092105,1
4,-1.0,13.0,-3.0,0.144737,1


In [3]:
# Defining the features and the target
X = data[['HeightDiff', 'WeightDiff', 'ReachDiff', 'WinDiff']]
y = data['Winner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,HeightDiff,WeightDiff,ReachDiff,WinDiff
3593,-2.0,-15.0,-1.0,0.183230
4353,-2.0,-20.0,-4.0,-0.135714
4660,4.0,0.0,3.0,-0.017647
2627,-1.0,0.0,-2.0,-0.196488
4118,5.0,0.0,-2.0,-0.154378


In [4]:
y_train.head()

3593    0
4353    0
4660    1
2627    1
4118    1
Name: Winner, dtype: int64

# Making PyTorch understand our data

Because PyTorch is a library that can use Tensors we are going to convert all of our data into tensors to speed up the process of training our model and also so we dont have to do it later.

In [5]:
# Convert training data into float tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float64)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float64)

# Making them long tensors since they are binary
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [6]:
# Making our dataset and dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Making the Neural Network Architecture

Given that our data is not very complex we will be using a simple neural network with 2 hidden layers and 1 output layer. We will be using the ReLU activation function for our hidden layers and the Sigmoid activation function for our output layer.

In [7]:
class Net(nn.Module):
    """
    Defining the neural network architecture with 2 hidden layers and 3 fully connected layers
    
    forward: like all neural networks, this function takes in the input and passes it through the layers
    """
    def __init__(self, input_size = 4, hidden_size_1 = 16, hidden_size_2 = 8, output_size = 2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # ReLU activation function
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        # Outputting using Sigmoid since we are using binary classification
        output = self.sigmoid(x)
        return output
    
    def loss(self, y_pred, y_true):
        # Binary cross entropy loss function
        criterion = nn.BCELoss()
        return criterion(y_pred, y_true)
    
    
# setting this variable for the future
net = Net()

# Optimizer Function and Setting Device

Setting the device so it will use the GPU is there is one, and also using the Adam optimizer due to our smaller dataset, and speed.

In [8]:
def optimizer(lr = 0.01):
    
    # Adam optimizer
    optimizer = optim.Adam(net.parameters(), lr=lr)
    return optimizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=2, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

# Training Loop

Now that we have most of our functions made we can now make our training loop. This is where we will be training our model and seeing how well it does.

In [12]:
def training(numb_epochs = 10):
    
    # Saving all epoch losses to visualize later
    epoch_losses = []
    
    # Training the model for how many times we say to
    for epoch in range(numb_epochs):
        
        # Saving the running loss
        running_loss = 0.0      
        
        for inputs, labels in train_loader:
            
            # Moving the data to the GPU (unless I am on my laptop)
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the gradients
            optimizer.zero_grad()
            # Foward pass
            outputs = net(inputs)
            # Calculating the loss
            loss = net.loss(outputs, labels)
            # Backward pass to compute the gradients
            loss.backward()
            # Updating Parameters
            optimizer.step()
            # Update Running loss for this epoch
            running_loss += loss.item()
            
        # Calculate the average loss for this epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_losses.append(epoch_loss)
            
        # Print the average loss for this epoch
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    
    # Plotting loss values
    plt.plot(range(1, numb_epochs + 1), epoch_losses, marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss vs. Epochs')
    plt.grid(True)
    plt.show()
    
    # DONNNNEEE
    print("Training finished.")